In [1]:
import graph_flow
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import ipywidgets
import IPython

In [3]:
N = 4
edges = [
    (0, 1, 90, 10),
    (0, 2, 100, 1),
    (1, 3, 110, 3),
    (2, 1, 130, 2),
    (2, 3, 140, 11)
]
g = graph_flow.Graph(N, edges)
G = nx.MultiDiGraph()
G.add_weighted_edges_from([(edge[0]+1, edge[1]+1, edge[3]) for edge in edges])

In [10]:
def show_flow(g: graph_flow.Graph, u: int, v: int, d: float):
    g.delete_flow()
    g.set_flow([(u, v, d)])
    found, cost, edges_flow = g.calculate_flow()
    EPS = 1e-9
    cmap = mpl.colormaps['inferno']
    edges_color = [1-max(edges_flow[i][0] / edge[2], 0) for i, edge in enumerate(edges)]
    pos = nx.spring_layout(G, seed=63)
    options = {
        "edge_color": edges_color,
        "edge_cmap": cmap,
        "edge_vmin": 0,
        "edge_vmax": 1.
    }
    nx.draw_networkx(G, pos, **options)
    plt.show()

In [11]:
def update(w=0):
    show_flow(g, 0, 3, w)
ipywidgets.interact(update, w=(0.0, 190.0, 0.1))
   

interactive(children=(FloatSlider(value=0.0, description='w', max=190.0), Output()), _dom_classes=('widget-int…

<function __main__.update(w=0)>

In [12]:
g.delete_flow()
g.set_flow([(0, 3, 120)])
g.calculate_flow()
g.block_edge(g.find_braess()[0])

In [13]:
ipywidgets.interact(update, w=(0.0, 190.0, 0.1))

interactive(children=(FloatSlider(value=0.0, description='w', max=190.0), Output()), _dom_classes=('widget-int…

<function __main__.update(w=0)>